In [ ]:
import pandas as pd
import ast

In [ ]:
df = pd.read_csv('../data/cleaned2.csv', low_memory=False)
df['categories'] = df['categories']\
    .apply(lambda x: ast.literal_eval(x))
df['mechanics'] = df['mechanics']\
    .apply(lambda x: ast.literal_eval(x))
df.head()

In [ ]:
def metric_by_year(data, metric):
    new_df = data[['year', metric]].copy()
    new_df = new_df.explode(metric).reset_index(drop=True)
    return new_df
    
cats_by_year = metric_by_year(df, 'categories')
mechs_by_year = metric_by_year(df, 'mechanics')
display(cats_by_year.head())
mechs_by_year.head()


In [ ]:
# Order metrics in descending order so we know which might be the interesting
# values to plot
def get_counts(data, metric):
    new_df = data[[metric]]
    new_df['val'] = 1
    new_df = new_df.groupby(metric).sum()
    return new_df

cat_counts = get_counts(cats_by_year, 'categories')
mech_counts = get_counts(mechs_by_year, 'mechanics')
display(cat_counts.sort_values('val', ascending=False).head(20))
display(mech_counts.sort_values('val', ascending=False).head(20))

In [ ]:
# Pivot the yearly data so it's plottable, and limit to years 1960-2023
def pivot(data, metric):
    new_df = data.copy()
    new_df['val'] = 1
    new_df = new_df.pivot_table(index='year', columns=metric, values='val', aggfunc='sum', fill_value=0)
    new_df = new_df.loc[new_df.index >= 1960].copy()
    new_df = new_df.loc[new_df.index < 2024].copy()
    return new_df

# Pivot and plot categories
cat_pivot = pivot(cats_by_year, 'categories')
cat_pivot[[
    'Card Game', 'War', "Children's Game", 'Party Game', 'Dice',
    'Abstract Strategy', 'Fantasy', 'Historical Setting', 'Educational',
    'Economy / Industry', 'Bluffing / Negotiation', 'Print & Play'
]].plot(title='Game Categories by Publication Year')

# Pivot and plot mechanics
mech_pivot = pivot(mechs_by_year, 'mechanics')
mech_pivot[[
    'Dice Rolling', 'Roll / Spin and Move', 'Hand Management', 'Set Collection',
    'Drafting', 'Simulation', 'Variable Player Powers', 'Cooperative Game',
    'Cooperative Game', 'Tile Placement', 'Grid-Based'
]].plot(title='Game Mechanics by Publication Year')